In [0]:
import sys
workspace = dbutils.widgets.get('workspace')
sys.path.insert(0, workspace)

from libs.ingestao_silver import ingestao_silver

from pyspark.sql.functions import col, cast, trim, when, to_date, upper, concat

In [0]:
catalog = 'projeto_cnpj'
table = 'estabelecimentos'
df = spark.table(f"{catalog}.bronze.{table}")

df= (df
    .withColumn('nome_fantasia', trim(upper(col('nome_fantasia'))))
    .withColumn('correio_eletronico', trim(upper(col('correio_eletronico'))))
    .withColumn('motivo_situacao_cadastral', col('motivo_situacao_cadastral').cast('int'))
    .withColumn('pais', col('pais').cast('int'))
    .withColumn('cnae_fiscal_principal', col('cnae_fiscal_principal').cast('int'))
    .withColumn('data_situacao_cadastral', 
                when(trim(col('data_situacao_cadastral')) != '0', to_date(col('data_situacao_cadastral'), 'yyyyMMdd'))
                .otherwise(None)
    )
    .withColumn('inicio_atividade', to_date(col('inicio_atividade'), 'yyyyMMdd'))
    .withColumn('data_situacao_especial', to_date(col('data_situacao_especial'), 'yyyyMMdd'))
    .withColumn('data_ingestao', col('data_ingestao').cast('date'))
    .withColumn('situacao_especial', trim(upper(col('situacao_especial'))))
    .withColumn('identificador',
        when(trim(col('identificador')) == '1', 'Matriz')
        .when(trim(col('identificador')) == '2', 'Filial')
        .otherwise('Outros')
    )
    .withColumn('situacao_cadastral',
        when(trim(col('situacao_cadastral')) == '08', 'Baixada')
        .when(trim(col('situacao_cadastral')) == '02', 'Ativa')
        .when(trim(col('situacao_cadastral')) == '03', 'Suspensa')
        .when(trim(col('situacao_cadastral')) == '00', 'Nula')
        .otherwise('Outros')
    )
    .filter(col('situacao_cadastral') != 'Outros')
)

In [0]:
ingestao_silver(
    spark = spark,
    df = df,
    catalog = catalog,
    table = table
)